# Pipeline completo: Preprocesado, Modelado y Evaluación en Google Colab
Este notebook ejecuta el flujo completo para clasificación de tumores cerebrales en imágenes MRI: descarga/preprocesado, entrenamiento de un modelo CNN y evaluación.

**Asegúrate de tener tu archivo `kaggle.json` para descargar el dataset desde Kaggle.**

## 1. Instalación de dependencias y configuración de entorno Colab

In [ ]:
# Instalar librerías necesarias
!pip install kaggle opencv-python-headless seaborn tensorflow scikit-learn --quiet
import os, shutil, zipfile, cv2, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

## 2. Descarga del dataset desde Kaggle

In [ ]:
# Subir kaggle.json manualmente
from google.colab import files
files.upload()  # Sube tu kaggle.json aquí
os.makedirs('/root/.kaggle', exist_ok=True)
shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 0o600)
!kaggle datasets download -d orvile/brain-cancer-mri-dataset -p /content/data --unzip

## 3. Reorganización y verificación de carpetas

In [ ]:
# Mover carpetas de clase a /content/data/original
os.makedirs('/content/data/original', exist_ok=True)
clases = ['brain_glioma', 'brain_menin', 'brain_tumor']
for clase in clases:
    origen = f'/content/data/brain-cancer-mri-dataset/{clase}'
    destino = f'/content/data/original/{clase}'
    if os.path.exists(origen) and not os.path.exists(destino):
        shutil.move(origen, destino)
# Eliminar carpeta vacía
try: shutil.rmtree('/content/data/brain-cancer-mri-dataset')
except: pass
# Verificar imágenes
for clase in clases:
    n = len(glob(f'/content/data/original/{clase}/*.jpg'))
    print(f'{clase}: {n} imágenes')

## 4. Preprocesado de imágenes y guardado

In [ ]:
def preprocesar_imagen(img_path, size=(224,224)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        with Image.open(img_path) as pil_img:
            img = np.array(pil_img.convert('L'))
    _, thresh = cv2.threshold(img, 5, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(thresh)
    if coords is not None:
        x, y, w, h = cv2.boundingRect(coords)
        img = img[y:y+h, x:x+w]
    img = cv2.resize(img, size)
    img = cv2.medianBlur(img, 5)
    img = cv2.equalizeHist(img)
    img = img.astype(np.float32)
    img = (img - img.min()) / (img.max() - img.min() + 1e-8)
    return img
# Guardar preprocesadas
os.makedirs('/content/data/preprocesadas', exist_ok=True)
for clase in clases:
    out_dir = f'/content/data/preprocesadas/{clase}'
    os.makedirs(out_dir, exist_ok=True)
    for img_path in glob(f'/content/data/original/{clase}/*.jpg'):
        img_proc = preprocesar_imagen(img_path)
        img_uint8 = (img_proc * 255).clip(0,255).astype(np.uint8)
        Image.fromarray(img_uint8).save(os.path.join(out_dir, os.path.basename(img_path)))

## 5. Preparación de arrays de datos y etiquetas

In [ ]:
X, y = [], []
clase_a_idx = {c: i for i, c in enumerate(clases)}
n_imagenes = 200  # Ajusta según recursos Colab
for clase in clases:
    imgs = glob(f'/content/data/preprocesadas/{clase}/*.jpg')[:n_imagenes]
    for img_path in imgs:
        img = preprocesar_imagen(img_path)
        X.append(img)
        y.append(clase_a_idx[clase])
X = np.array(X)[..., np.newaxis]
y = np.array(y)
print('Shape X:', X.shape)
print('Shape y:', y.shape)

## 6. División en train, val y test

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)
print(f'Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}')

## 7. Modelado: Definición y entrenamiento de una CNN simple

In [ ]:
from tensorflow.keras import layers, models
input_shape = X_train.shape[1:]
num_classes = len(clases)
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 8. Entrenamiento del modelo

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

## 9. Evaluación y visualización de resultados

In [ ]:
# Evaluación en test
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión en test: {test_acc:.4f}')
# Matriz de confusión
y_pred = np.argmax(model.predict(X_test), axis=1)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=clases, yticklabels=clases)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de confusión')
plt.show()
# Reporte de clasificación
print(classification_report(y_test, y_pred, target_names=clases))